In [2]:
import import_ipynb
import data_preprocessing

- remove seasonablity
- drop nans (if pre-processing from seasonality)
- others?

In [4]:
# training data
u4_train_equil_turbine = data_preprocessing.u4_train_equil_turbine
u4_train_equil_pump = data_preprocessing.u4_train_equil_pump
u5_train_equil_turbine = data_preprocessing.u5_train_equil_turbine
u5_train_equil_pump = data_preprocessing.u5_train_equil_pump
u6_train_equil_turbine = data_preprocessing.u6_train_equil_turbine
u6_train_equil_pump = data_preprocessing.u6_train_equil_pump

# synethetic test sets
u5_s01_equil_turbine = data_preprocessing.u5_s01_equil_turbine
u5_s01_equil_pump = data_preprocessing.u5_s01_equil_pump
u5_s02_equil_turbine = data_preprocessing.u5_s02_equil_turbine
u5_s02_equil_pump = data_preprocessing.u5_s02_equil_pump
u6_s01_equil_turbine = data_preprocessing.u6_s01_equil_turbine
u6_s01_equil_pump = data_preprocessing.u6_s01_equil_pump
u6_s02_equil_turbine = data_preprocessing.u6_s02_equil_turbine
u6_s02_equil_pump = data_preprocessing.u6_s02_equil_pump

# real test sets
u4_test_equil_turbine = data_preprocessing.u4_test_equil_turbine
u4_test_equil_pump = data_preprocessing.u4_test_equil_pump
u5_test_equil_turbine = data_preprocessing.u5_test_equil_turbine
u5_test_equil_pump = data_preprocessing.u5_test_equil_pump
u6_test_equil_turbine = data_preprocessing.u6_test_equil_turbine
u6_test_equil_pump = data_preprocessing.u6_test_equil_pump

In [ ]:
u4_train_equil_pump

,tot_activepower,plant_tmp,ext_tmp,water_primary_cold_tmp,water_primary_hot_tmp,valve_opening,refri_bath_level,aspi_bath_level,canal_level,canal_tmp,...,water_circ_hot_02_tmp,water_circ_cold_tmp,machine_on,machine_off,turbine_mode,all,equilibrium_turbine_mode,dyn_only_on,pump_mode,equilibrium_pump_mode
2020-01-02 00:00:30+01:00,-62.433334,16.554980,5.992895,10.850000,12.914706,99.873437,2.955667,5.927700,371.836559,8.912023,...,16.797366,12.095122,True,False,False,True,False,True,True,True
2020-01-02 00:01:00+01:00,-62.571904,16.593311,6.024210,10.800000,13.002564,99.871875,2.960449,5.922808,371.711225,8.824047,...,16.712043,11.969360,True,False,False,True,False,True,True,True
2020-01-02 00:01:30+01:00,-62.507894,16.631641,6.055526,10.850000,13.079487,99.870312,2.968310,5.917917,371.765866,8.736070,...,16.626720,11.825357,True,False,False,True,False,True,True,True
2020-01-02 00:02:00+01:00,-62.397368,16.669971,6.086841,10.900000,13.126829,99.868750,2.979270,5.913025,371.842773,8.648094,...,16.369280,11.608393,True,False,False,True,False,True,True,True
2020-01-02 00:02:30+01:00,-62.451427,16.708301,6.118156,10.900000,13.163415,99.867188,2.989435,5.908133,371.902988,8.560117,...,16.097873,11.391429,True,False,False,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-29 03:00:30+01:00,-62.414286,18.422242,9.134391,15.876000,20.900000,55.667491,2.899200,5.796250,371.973697,7.000000,...,20.127665,15.927082,True,False,False,True,False,False,True,True
2020-12-29 03:01:00+01:00,-62.369999,18.436623,9.137895,15.756000,20.900000,55.910297,2.905200,5.788500,371.987273,7.000000,...,20.089751,15.924746,True,False,False,True,False,False,True,True
2020-12-29 03:01:30+01:00,-62.486706,18.451003,9.069133,15.554545,20.843396,56.153103,2.912571,5.780750,372.000849,7.000000,...,20.051837,15.922410,True,False,False,True,False,False,True,True
2020-12-29 03:02:00+01:00,-62.529059,18.465383,8.964238,15.348148,20.870000,56.395908,2.921143,5.773000,372.014425,7.000000,...,20.013924,15.920074,True,False,False,True,False,False,True,True
